In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2025/Conferences.csv
/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-20

In [3]:
# Team names, IDs; conferences
MTeams = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MTeams.csv')
WTeams = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WTeams.csv')

MTeamConfs = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MTeamConferences.csv')
WTeamConfs = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WTeamConferences.csv')


# Dates and MM regions
# Day 132 is Selection Sunday and Day 154 is men's championship game
MSeasons = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv')
WSeasons = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WSeasons.csv')

# Seeds for MM, regions are W-Z and double seeds have a's or b's'
MSeeds = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeeds.csv')
WSeeds = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySeeds.csv')

# Regular season results (day 132 or earlier), WLoc is location of winning team
# Box scores are in the detail files
MRegSzn = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv')
MRegSznDetail = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv')
MConfTourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MConferenceTourneyGames.csv')

WRegSzn = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv')
WRegSznDetail = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonDetailedResults.csv')
WConfTourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WConferenceTourneyGames.csv')

# Tournament results (after day 132)
MTourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv')
MTourneyDetail = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyDetailedResults.csv')
MSlots = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySlots.csv')

WTourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv')
WTourneyDetail = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv')
WSlots = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv')


# Rankings (not all systems necessarily available this year)
MRanks = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MMasseyOrdinals.csv')
SeedBenchmarks = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv')


In [3]:
t_teams = pd.merge(MSeeds, MTeams[['TeamID','TeamName']], on='TeamID')
t_teams['RSeed'] = t_teams['Seed'].str[1:3]
t_teams['Region'] = t_teams['Seed'].str[0]
t_teams['Seed'] = t_teams['Seed'].str[1:]

results = pd.merge(MTourney, t_teams, left_on=['WTeamID','Season'],right_on=['TeamID','Season'])
results = pd.merge(results, t_teams, left_on=['LTeamID','Season'],right_on=['TeamID','Season'])

results = results.drop(columns=['TeamID_x', 'TeamID_y', 'Seed_x','Seed_y','NumOT', 'WLoc','Region_x'])
results.columns = ['year','day','WID','WScore','LID','LScore','WTeam','WSeed','LTeam','LSeed','Region']

results.head()

,year,day,WID,WScore,LID,LScore,WTeam,WSeed,LTeam,LSeed,Region
0,1985,136,1116,63,1234,54,Arkansas,09,Iowa,08,X
1,1985,136,1120,59,1345,58,Auburn,11,Purdue,06,Z
2,1985,136,1207,68,1250,43,Georgetown,01,Lehigh,16,W
3,1985,136,1229,58,1425,55,Illinois St,09,USC,08,Y
4,1985,136,1242,49,1325,38,Kansas,03,Ohio,14,Z


In [61]:
#results.groupby(['WSeed','LSeed'])['year'].count().reset_index().sort_values('year',ascending=False)

r_box_raw = MRegSznDetail
t_box_raw = MTourneyDetail

def reshape_box_scores(df):
    winning_stat_cols = [col for col in df.columns if col.startswith('W') and col != 'WTeamID' and col != 'WLoc']
    losing_stat_cols = [col for col in df.columns if col.startswith('L') and col != 'LTeamID']
    
    # Create two separate dataframes - one for winning teams and one for losing teams
    winning_df = df[['WTeamID'] + winning_stat_cols].copy()
    winning_df.columns = [col.replace('W', 'For') if col.startswith('W') else 'team_id' 
                          for col in winning_df.columns]
    winning_df['win'] = 1


    print(winning_df.shape)
    
    # For losing teams
    losing_df = df[['LTeamID'] + losing_stat_cols].copy()
    losing_df.columns = [col.replace('L', 'All') if col.startswith('L') else 'team_id' 
                         for col in losing_df.columns]
    losing_df['loss'] = 0

    print(losing_df.shape)
    
    first_half = pd.concat([r_box_raw[['Season', 'DayNum']], winning_df, losing_df], axis=1)
    second_half = pd.concat([r_box_raw[['Season', 'DayNum']], losing_df, winning_df], axis=1)

    second_half.columns = first_half.columns

    return pd.concat([first_half, second_half], ignore_index=True)

r_box_scores = reshape_box_scores(r_box_raw)
r_box_scores.head(10)

(118449, 16)
(118449, 16)


,Season,DayNum,ForTeamID,ForScore,ForFGM,ForFGA,ForFGM3,ForFGA3,ForFTM,ForFTA,...,AllFTM,AllFTA,AllOR,AllDR,AllAst,AllTO,AllStl,AllBlk,AllPF,loss
0,2003,10,1104,68,27,58,3,14,11,18,...,16,22,10,22,8,18,9,2,20,0
1,2003,10,1272,70,26,62,8,20,10,19,...,9,20,20,25,7,12,8,6,16,0
2,2003,11,1266,73,24,58,8,18,17,29,...,14,23,31,22,9,12,2,5,23,0
3,2003,11,1296,56,18,38,3,9,17,31,...,8,15,17,20,9,19,4,3,23,0
4,2003,11,1400,77,30,61,6,14,11,13,...,17,27,21,15,12,10,7,1,14,0
5,2003,11,1458,81,26,57,6,12,23,27,...,12,17,6,22,8,19,4,3,25,0
6,2003,12,1161,80,23,55,2,8,32,39,...,20,28,9,21,11,30,10,4,28,0
7,2003,12,1186,75,28,62,4,14,15,21,...,17,23,8,25,10,15,14,8,18,0
8,2003,12,1194,71,28,58,5,11,10,18,...,12,27,13,26,13,25,8,2,18,0
9,2003,12,1458,84,32,67,5,17,15,19,...,7,12,9,23,10,18,1,3,18,0


In [83]:
team_stats = r_box_scores.groupby(['ForTeamID','Season']).mean().reset_index()
for col in team_stats.columns:
    team_stats[col] = team_stats[col].round(2)

team_games = r_box_scores.groupby(['ForTeamID', 'Season'])['DayNum'].count().reset_index()
team_games.columns = ['ForTeamID', 'Season', 'NGames']

team_stats = pd.merge(MTeams[['TeamID', 'TeamName']], team_stats, right_on='ForTeamID', left_on='TeamID')
team_stats = pd.merge(team_stats, team_games, on=['ForTeamID','Season'])

team_stats = team_stats.drop(columns=['DayNum', 'loss', 'TeamID']).sort_values('win', ascending=False)
team_stats.head(10)

,TeamName,ForTeamID,Season,ForScore,ForFGM,ForFGA,ForFGM3,ForFGA3,ForFTM,ForFTA,...,AllFTM,AllFTA,AllOR,AllDR,AllAst,AllTO,AllStl,AllBlk,AllPF,NGames
3124,Kentucky,1246,2015,74.91,25.97,55.41,5.44,15.56,17.53,24.26,...,11.09,17.09,11.71,19.03,7.68,14.06,4.68,2.32,19.91,34
7700,Wichita St,1455,2014,74.94,24.73,54.09,6.85,19.73,18.64,25.58,...,12.30,18.88,9.36,21.64,10.06,11.97,4.67,2.61,21.09,33
2381,Gonzaga,1211,2021,92.12,34.15,62.00,7.31,20.04,16.50,22.73,...,11.12,15.81,7.19,20.88,11.46,14.46,6.00,2.88,20.12,26
3701,Memphis,1272,2008,79.68,28.76,61.76,7.94,22.68,14.21,23.85,...,12.47,18.53,11.03,23.24,10.62,16.59,5.91,3.21,19.41,34
2377,Gonzaga,1211,2017,84.58,30.36,58.64,7.30,19.12,16.55,22.42,...,10.42,15.85,10.52,22.00,9.64,12.52,5.21,2.64,19.61,33
6264,Stanford,1390,2004,73.93,26.53,54.80,5.20,14.20,15.67,20.83,...,11.83,17.07,10.40,19.40,10.20,14.70,6.00,2.07,19.77,30
2714,Illinois,1228,2005,77.42,28.15,57.61,8.70,21.79,12.42,17.15,...,10.27,15.52,10.27,20.52,12.30,15.30,4.73,2.36,15.55,33
7753,Winthrop,1457,2021,79.54,28.08,60.96,8.46,23.96,14.92,21.75,...,12.42,18.75,6.08,21.62,10.38,15.83,6.00,3.33,22.00,24
6172,St Joseph's PA,1386,2004,77.79,27.18,56.79,9.75,23.96,13.68,19.46,...,12.86,18.68,13.64,23.04,11.64,16.57,6.71,2.39,17.96,28
4166,Murray St,1293,2012,74.21,24.64,51.82,7.43,18.25,17.50,24.04,...,12.61,18.96,12.07,20.00,10.50,16.04,6.46,2.71,20.46,28


In [ ]:
# Calculate test Brier score
def brier_score(df, prob_col, result_col):
    return np.mean((df[prob_col] - df[result_col]) ** 2)
    
score = brier_score(df, df['Pred'], df['Result'])
print(f"Brier Score: {score:.4f}")